In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split

from tensorflow import keras
from keras import layers
from keras import callbacks

path = "/content/drive/MyDrive/dataset/PDOs PY copy.csv"
data = pd.read_csv(path)

X = data.copy()
y = X.pop('PY')

features_num = ['Initial glycerol concentration (M)', 'Stir rate (rpm)',
                'Anode surface area (cm2)', 'Cathode surface area (cm2)',
                'T (deg C)', 'J (A/cm2)', 'pH', 'Time']
features_cat = ['Feed', 'Cell type', 'Anode', 'Cathode', 'Electrolyte']

preprocessor = make_column_transformer(
    (StandardScaler(), features_num),
    (OneHotEncoder(), features_cat),
)

X_train, X_valid, y_train, y_valid = train_test_split(X,y)

X_train = preprocessor.fit_transform(X_train)
X_valid = preprocessor.transform(X_valid)

#Get list of categorical variables
# s = (X.dtypes == 'object')
# object_cols = list(s[s].index)

#preprocessor = make_column_transformer(
#     (StandardScaler(),
#      make_column_selector(dtype_include=np.number)),
#     (OneHotEncoder(handle_unknown='ignore', sparse_output=False),
#      make_column_selector(dtype_include=object)),
# )

# OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
# OH_cols = pd.DataFrame(OH_encoder.fit_transform(X[object_cols]))
# OH_cols.index = X.index
# num_X = X.drop(object_cols, axis=1)
# OH_X = pd.concat([num_X, OH_cols], axis=1)
# OH_X.columns = OH_X.columns.astype(str)

#X = preprocessor.fit_transform(X)
#y = np.log(y) # log transform target instead of standardizing

input_shape = [X_train.shape[1]]
print("Input shape: {}".format(input_shape))

Input shape: [30]


In [ ]:
data.head()


,Feed,Initial glycerol concentration (M),Cell type,Stir rate (rpm),Anode,Cathode,Anode surface area (cm2),Cathode surface area (cm2),T (deg C),J (A/cm2),V (V),pH,Electrolyte,Time,PY
0,Glycerol,0.5,Single cell,400,Pt,Pt,66.49,124.34,30,0.036191,2.0,1,H2SO4,6,60.0
1,Glycerol,0.3,Single cell,650,Pt,Pt,121.38,31.82,25,0.140000,0.0,1,H2SO4,0,0.0
2,Glycerol,0.3,Single cell,650,Pt,Pt,121.38,31.82,25,0.140000,0.0,1,H2SO4,1,0.0
3,Glycerol,0.3,Single cell,650,Pt,Pt,121.38,31.82,25,0.140000,0.0,1,H2SO4,2,0.0
4,Glycerol,0.3,Single cell,650,Pt,Pt,121.38,31.82,25,0.140000,0.0,1,H2SO4,3,0.0


In [ ]:
pd.DataFrame(X_train[:10,:]).head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-2.214145,-0.603421,-1.659446,-0.854507,-0.282342,-1.683795,1.222495,-0.267403,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.451642,-0.603421,0.636774,-1.725239,-0.282342,0.153827,-0.538480,-0.981658,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.451642,-0.603421,0.636774,0.793550,-0.282342,1.466414,-0.538480,-0.410254,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-2.214145,-0.603421,-1.659446,-0.854507,-0.282342,-1.683795,-0.538480,-0.267403,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-2.214145,-0.603421,-1.659446,-0.854507,-0.282342,-1.683795,-0.538480,-0.838807,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
model = keras.Sequential([
    #layers.BatchNormalization(input_shape=input_shape),
    layers.Dense(512, activation='relu', input_shape=input_shape),
    #layers.BatchNormalization(),
    #layers.Dropout(0.3),
    layers.Dense(512, activation='relu'),
    #layers.BatchNormalization(),
    #layers.Dropout(0.3),
    layers.Dense(512, activation='relu'),
    #layers.BatchNormalization(),
    #layers.Dropout(0.3),
    layers.Dense(1)
])

model.compile(
optimizer='adam',
loss='mae',
)

history = model.fit(
  X_train, y_train,
  validation_data=(X_valid,y_valid),
  batch_size=64,
  epochs=50,
)


Epoch 1/50
4/4 [==============================] - 2s 191ms/step - loss: 3.7795 - val_loss: 6.9030
Epoch 2/50
4/4 [==============================] - 0s 24ms/step - loss: 3.4084 - val_loss: 6.5248
Epoch 3/50
4/4 [==============================] - 0s 23ms/step - loss: 3.1768 - val_loss: 6.2859
Epoch 4/50
4/4 [==============================] - 0s 28ms/step - loss: 2.9071 - val_loss: 6.0563
Epoch 5/50
4/4 [==============================] - 0s 28ms/step - loss: 2.8210 - val_loss: 5.9167
Epoch 6/50
4/4 [==============================] - 0s 22ms/step - loss: 2.7188 - val_loss: 5.8050
Epoch 7/50
4/4 [==============================] - 0s 31ms/step - loss: 2.6980 - val_loss: 5.6530
Epoch 8/50
4/4 [==============================] - 0s 32ms/step - loss: 2.5613 - val_loss: 5.5287
Epoch 9/50
4/4 [==============================] - 0s 33ms/step - loss: 2.4654 - val_loss: 5.3292
Epoch 10/50
4/4 [==============================] - 0s 27ms/step - loss: 2.4001 - val_loss: 5.2222
Epoch 11/50
4/4 [===========

In [ ]:
from sklearn import metrics
from sklearn.metrics import r2_score

pred = model.predict(X_valid)

score = metrics.mean_squared_error(pred, y_valid)
print("Final score (MSE): {}".format(score))

r2score = r2_score(y_valid, pred)
print("Final score (R2 score): {}".format(r2score))

3/3 [==============================] - 0s 6ms/step
Final score (MSE): 52.28341915490235
Final score (R2 score): 0.6668080935318488


In [ ]:
import numpy as np

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_valid))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 7.230727429166608


In [ ]:
#########improved version always second (CR done)
!pip install keras-adamw
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt
from keras.optimizers import AdamW
from tensorflow import keras
from keras import layers
from keras import callbacks

path = "/content/drive/MyDrive/dataset/raw data.csv" #/content/drive/MyDrive/dataset/CR.csv
data = pd.read_csv(path)

data = pd.get_dummies(data) #convert categorical variable into dummy variable

y = np.array(data['CR (%)']) #output feature
X = data.drop(['CR (%)', 'PDOs PY (%)', 'I (A)'], axis=1) #input feature - axis=1 refers to the column

X_list = list(X.columns) # Saving feature names for later use

from sklearn.preprocessing import StandardScaler #standardscaler for normalization (Jinesh et al., 2023)
scaler = StandardScaler()

y = y.reshape(-1, 1) #normalizing y
scaler.fit(y)
y = scaler.transform(y)

scaler.fit(X) #normalizing X
X = scaler.transform(X)

X = np.array(X) # Convert to numpy array

# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42)
print('Training X Shape:', train_X.shape)
print('Training y Shape:', train_y.shape)
print('Testing X Shape:', test_X.shape)
print('Testing y Shape:', test_y.shape)

#imputation for missing value using kNN algo
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10) #create an instance of KNNImputer class, k is desired number of neighbours

#perform imputation only on X because y no missing values
imp_train_X = imputer.fit_transform(train_X)
imp_test_X = imputer.fit_transform(test_X)

input_shape = [imp_train_X.shape[1]]


model = keras.Sequential([
    layers.Dense(125, activation='relu', input_shape=input_shape),
    #layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(125, activation='relu'),
    #layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(125, activation='relu'),
    #layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(1)
])

# adamw = AdamW(lr=0.0001, weight_decay=1e-5)
# model.compile(optimizer=adamw, loss='categorical_crossentropy', metrics=['accuracy'])

model.compile(
optimizer='adam',
loss='mae',
)

history = model.fit(
  imp_train_X, train_y,
  validation_data=(imp_test_X,test_y),
  batch_size=3,
  epochs=1000,
)

#r2 score for training set
y_pred = model.predict(imp_train_X)
y_true = train_y
score = r2_score(y_true,y_pred)
print("Training score: ", score)

pred = model.predict(imp_test_X)

print("Mean Absolute Error: " + str(mean_absolute_error(test_y, pred)))
print("MSE: " + str(mean_squared_error(pred, test_y)))

RMSE = sqrt(mean_squared_error(pred, test_y))

print("RMSE: " + str(RMSE))
print("R2 score: " + str(r2_score(test_y, pred)))


Training X Shape: (312, 30)
Training y Shape: (312, 1)
Testing X Shape: (134, 30)
Testing y Shape: (134, 1)
Epoch 1/1000
104/104 [==============================] - 8s 12ms/step - loss: 0.7513 - val_loss: 0.5455
Epoch 2/1000
104/104 [==============================] - 1s 7ms/step - loss: 0.5759 - val_loss: 0.4211
Epoch 3/1000
104/104 [==============================] - 1s 7ms/step - loss: 0.5109 - val_loss: 0.3954
Epoch 4/1000
104/104 [==============================] - 1s 8ms/step - loss: 0.4771 - val_loss: 0.4356
Epoch 5/1000
104/104 [==============================] - 1s 7ms/step - loss: 0.4444 - val_loss: 0.2861
Epoch 6/1000
104/104 [==============================] - 1s 8ms/step - loss: 0.4249 - val_loss: 0.2828
Epoch 7/1000
104/104 [==============================] - 1s 9ms/step - loss: 0.4020 - val_loss: 0.3158
Epoch 8/1000
104/104 [==============================] - 1s 10ms/step - loss: 0.3626 - val_loss: 0.2949
Epoch 9/1000
104/104 [==============================] - 1s 13ms/step - los

In [ ]:
#sometime wait for the third will not be too bad!

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

from tensorflow import keras
from keras import layers
from keras import callbacks

path = "/content/drive/MyDrive/dataset/PDOs PY copy.csv"
data = pd.read_csv(path)

data = pd.get_dummies(data) #convert categorical variable into dummy variable

y = np.array(data['PY']) #output feature
X = data.drop(['PY'], axis=1) #input feature - axis=1 refers to the column

X_list = list(X.columns) # Saving feature names for later use

from sklearn.preprocessing import StandardScaler #standardscaler for normalization (Jinesh et al., 2023)
scaler = StandardScaler()

y = y.reshape(-1, 1) #normalizing y
scaler.fit(y)
y = scaler.transform(y)

scaler.fit(X) #normalizing X
X = scaler.transform(X)

X = np.array(X) # Convert to numpy array

# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.25, random_state = 42)
print('Training X Shape:', train_X.shape)
print('Training y Shape:', train_y.shape)
print('Testing X Shape:', test_X.shape)
print('Testing y Shape:', test_y.shape)

#imputation for missing value using kNN algo
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10) #create an instance of KNNImputer class, k is desired number of neighbours

#perform imputation on on X because y no missing values
imp_train_X = imputer.fit_transform(train_X)
imp_test_X = imputer.fit_transform(test_X)

input_shape = [imp_train_X.shape[1]]

model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=input_shape),
    #layers.BatchNormalization(),
    #layers.Dropout(0.3),
    layers.Dense(512, activation='relu'),
    #layers.BatchNormalization(),
    #layers.Dropout(0.3),
    #layers.Dense(512, activation='relu'),
    #layers.BatchNormalization(),
    #layers.Dropout(0.3),
    layers.Dense(1)
])

model.compile(
optimizer='adam',
loss='mae')

r = model.fit(imp_train_X, train_y,
              validation_data=(imp_test_X,test_y),
              batch_size=64,
              epochs=50)

#r2 score for training set (true, predicted)
test_pred = model.predict(imp_test_X)
train_pred = model.predict(imp_train_X)

score = r2_score(train_y, train_pred)
print("Training R2 score: ", score)

print("Mean Absolute Error: " + str(mean_absolute_error(test_y, test_pred)))
print("MSE: " + str(mean_squared_error(test_y, test_pred)))

RMSE = sqrt(mean_squared_error(test_y, test_pred))

print("RMSE: " + str(RMSE))
print("R2 score: " + str(r2_score(test_y, test_pred)))



Training X Shape: (242, 31)
Training y Shape: (242, 1)
Testing X Shape: (81, 31)
Testing y Shape: (81, 1)
Epoch 1/50
4/4 [==============================] - 1s 61ms/step - loss: 0.6235 - val_loss: 0.4573
Epoch 2/50
4/4 [==============================] - 0s 24ms/step - loss: 0.4789 - val_loss: 0.4533
Epoch 3/50
4/4 [==============================] - 0s 22ms/step - loss: 0.4495 - val_loss: 0.3864
Epoch 4/50
4/4 [==============================] - 0s 17ms/step - loss: 0.4360 - val_loss: 0.4190
Epoch 5/50
4/4 [==============================] - 0s 23ms/step - loss: 0.3776 - val_loss: 0.4184
Epoch 6/50
4/4 [==============================] - 0s 24ms/step - loss: 0.3927 - val_loss: 0.3544
Epoch 7/50
4/4 [==============================] - 0s 17ms/step - loss: 0.3485 - val_loss: 0.4084
Epoch 8/50
4/4 [==============================] - 0s 23ms/step - loss: 0.3318 - val_loss: 0.3525
Epoch 9/50
4/4 [==============================] - 0s 18ms/step - loss: 0.3292 - val_loss: 0.3082
Epoch 10/50
4/4 [====

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow

Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.12.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? Y
  Successfully uninstalled tensorflow-2.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.8 MB/s eta 0:00:00
  Attempting uninstall: tenso

In [ ]:
#fourth? OK! gridsearch

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
from math import sqrt

path = "/content/drive/MyDrive/dataset/raw data.csv"
data = pd.read_csv(path)

data = pd.get_dummies(data) #convert categorical variable into dummy variable

y = np.array(data['CR (%)']) #output feature
X = data.drop(['CR (%)', 'PDOs PY (%)', 'I (A)'], axis=1) #input feature - axis=1 refers to the column

X_list = list(X.columns) # Saving feature names for later use

from sklearn.preprocessing import StandardScaler #standardscaler for normalization (Jinesh et al., 2023)
scaler = StandardScaler()

y = y.reshape(-1, 1) #normalizing y
scaler.fit(y)
y = scaler.transform(y)

scaler.fit(X) #normalizing X
X = scaler.transform(X)

X = np.array(X) # Convert to numpy array

# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42)
print('Training X Shape:', train_X.shape)
print('Training y Shape:', train_y.shape)
print('Testing X Shape:', test_X.shape)
print('Testing y Shape:', test_y.shape)

#imputation for missing value using kNN algo
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10) #create an instance of KNNImputer class, k is desired number of neighbours

#perform imputation on on X because y no missing values
imp_train_X = imputer.fit_transform(train_X)
imp_test_X = imputer.fit_transform(test_X)

# Function to create the Keras regressor model
def create_model(hidden_units=50, activation='relu', optimizer='adam'):
    model = Sequential()
    model.add(Dense(hidden_units, activation=activation, input_shape=(imp_train_X.shape[1],)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

# Create the KerasRegressor model
model = KerasRegressor(model=create_model, verbose=0)

# Define the hyperparameters and their values to search over
param_grid = {
    #'hidden_units': [50, 100, 200], #for 50 r2 train=0.865, r2 test=0.795 #for 100 r2 train=0.888, r2 test=0.903 #for 100 r2 train=0.888, r2 test=0.911
    #'activation': ['relu', 'tanh'],
    'optimizer': ['adam', 'sgd'],
    'epochs': [100, 250, 1000], #>300 underfit, 250 is best
    'batch_size': [3, 8, 16]
}

# Create the GridSearchCV object with the KerasRegressor model and the parameter grid
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# Fit the GridSearchCV object to your data
grid_result = grid.fit(imp_train_X, train_y)

# Get the best hyperparameters and the best model
best_params = grid_result.best_params_
best_model = grid_result.best_estimator_
print('Best hyperparameters:', best_params)

# Evaluate the best model on your test data
train_pred = best_model.predict(imp_train_X)
test_pred = best_model.predict(imp_test_X)

score = r2_score(train_y, train_pred)
print("Training R2 score: ", score)

print("Mean Absolute Error: " + str(mean_absolute_error(test_y, test_pred)))
print("MSE: " + str(mean_squared_error(test_y, test_pred)))

RMSE = sqrt(mean_squared_error(test_y, test_pred))

print("RMSE: " + str(RMSE))
print("R2 score: " + str(r2_score(test_y, test_pred)))


ImportError: cannot import name 'KerasRegressor' from 'keras' (/usr/local/lib/python3.10/dist-packages/keras/__init__.py)

In [ ]:
print(model.get_params().keys())


dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs'])


In [2]:
########improved version always second for PDOs PY now ECR
!pip install keras-adamw
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt
from keras.optimizers import AdamW
from tensorflow import keras
from keras import layers
from keras import callbacks

path = "/content/drive/MyDrive/dataset/raw data.csv" #/content/drive/MyDrive/dataset/CR.csv
data = pd.read_csv(path)

data = pd.get_dummies(data) #convert categorical variable into dummy variable

y = np.array(data['PDOs PY (%)']) #output feature
X = data.drop(['CR (%)', 'ECR PY (%)', 'PDOs PY (%)', 'POHs PY (%)', 'V'], axis=1) #input feature - axis=1 refers to the column

X_list = list(X.columns) # Saving feature names for later use

from sklearn.preprocessing import StandardScaler #standardscaler for normalization (Jinesh et al., 2023)
scaler = StandardScaler()

y = y.reshape(-1, 1) #normalizing y
scaler.fit(y)
y = scaler.transform(y)

scaler.fit(X) #normalizing X
X = scaler.transform(X)

X = np.array(X) # Convert to numpy array

# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42)
print('Training X Shape:', train_X.shape)
print('Training y Shape:', train_y.shape)
print('Testing X Shape:', test_X.shape)
print('Testing y Shape:', test_y.shape)

#imputation for missing value using kNN algo
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10) #create an instance of KNNImputer class, k is desired number of neighbours

#perform imputation only on X because y no missing values
imp_train_X = imputer.fit_transform(train_X)
imp_test_X = imputer.fit_transform(test_X)

input_shape = [imp_train_X.shape[1]]


model = keras.Sequential([
    layers.Dense(125, activation='relu', input_shape=input_shape),
    #layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(125, activation='relu'),
    #layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(125, activation='relu'),
    #layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(1)
])

# adamw = AdamW(lr=0.0001, weight_decay=1e-5)
# model.compile(optimizer=adamw, loss='categorical_crossentropy', metrics=['accuracy'])

model.compile(
optimizer='adam',
loss='mae',
)

history = model.fit(
  imp_train_X, train_y,
  validation_data=(imp_test_X,test_y),
  batch_size=3,
  epochs=1000,
)

#r2 score for training set
y_pred = model.predict(imp_train_X)
y_true = train_y
score = r2_score(y_true,y_pred)
print("Training score: ", score)

pred = model.predict(imp_test_X)

print("Mean Absolute Error: " + str(mean_absolute_error(test_y, pred)))
print("MSE: " + str(mean_squared_error(pred, test_y)))

RMSE = sqrt(mean_squared_error(pred, test_y))

print("RMSE: " + str(RMSE))
print("R2 score: " + str(r2_score(test_y, pred)))

Training X Shape: (312, 30)
Training y Shape: (312, 1)
Testing X Shape: (134, 30)
Testing y Shape: (134, 1)
Epoch 1/1000
104/104 [==============================] - 4s 12ms/step - loss: 0.6112 - val_loss: 0.6210
Epoch 2/1000
104/104 [==============================] - 1s 8ms/step - loss: 0.5597 - val_loss: 0.6350
Epoch 3/1000
104/104 [==============================] - 1s 10ms/step - loss: 0.5212 - val_loss: 0.5590
Epoch 4/1000
104/104 [==============================] - 1s 6ms/step - loss: 0.4804 - val_loss: 0.5561
Epoch 5/1000
104/104 [==============================] - 1s 5ms/step - loss: 0.4931 - val_loss: 0.5687
Epoch 6/1000
104/104 [==============================] - 1s 6ms/step - loss: 0.4672 - val_loss: 0.5354
Epoch 7/1000
104/104 [==============================] - 1s 6ms/step - loss: 0.4278 - val_loss: 0.4919
Epoch 8/1000
104/104 [==============================] - 1s 5ms/step - loss: 0.4252 - val_loss: 0.4744
Epoch 9/1000
104/104 [==============================] - 1s 5ms/step - loss